In [1]:
# 📘 02_Werkvertrag_OCR_Analyse.ipynb

# Schritt 1: Benötigte Bibliotheken importieren
import pytesseract
from pdf2image import convert_from_bytes
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, Markdown

# Schritt 2: Datei-Upload-Widget anzeigen
upload = widgets.FileUpload(accept='.pdf', multiple=False)
display(Markdown("## 📂 PDF-Datei des Werkvertrags hochladen:"))
display(upload)

# Schritt 3: Funktion zur OCR-Verarbeitung definieren
def run_ocr(uploaded_file):
    # Die hochgeladene Datei liegt als tuple in .value vor
    uploaded_data = uploaded_file.value
    if not uploaded_data:
        return ""
    
    # Hole den Inhalt aus dem Tupel
    file_info = uploaded_data[0]  # erstes (und einziges) Element
    file_content = file_info['content']
    
    # PDF-Seiten in Bilder umwandeln
    images = convert_from_bytes(file_content)

    # OCR auf jeder Seite durchführen
    full_text = ""
    for i, img in enumerate(images):
        text = pytesseract.image_to_string(img, lang='deu')
        full_text += f"\n--- Seite {i+1} ---\n{text}\n"

    return full_text


# Schritt 4: Analysefunktion für Werkvertrags-Inhalte (einfaches Beispiel)
def extract_key_data(text):
    result = {
        "Vertragspartner": "",
        "Datum": "",
        "Werkpreis": "",
        "Zahlungskonditionen": "",
        "Ausführung von/bis": "",
        "Unterschriften": ""
    }
    
    # Suche nach typischen Begriffen (Beispielhaft)
    if "Vertrag zwischen" in text:
        result["Vertragspartner"] = text.split("Vertrag zwischen",1)[1].split("\n",1)[0].strip()
    if "Datum" in text:
        result["Datum"] = text.split("Datum",1)[1].split("\n",1)[0].strip()
    if "Werkpreis" in text:
        result["Werkpreis"] = text.split("Werkpreis",1)[1].split("\n",1)[0].strip()
    if "Zahlung" in text:
        result["Zahlungskonditionen"] = text.split("Zahlung",1)[1].split("\n",1)[0].strip()
    if "Baubeginn" in text:
        result["Ausführung von/bis"] = text.split("Baubeginn",1)[1].split("\n",1)[0].strip()
    if "Unterschrift" in text:
        result["Unterschriften"] = "gefunden"

    return pd.DataFrame([result])

# Schritt 5: Schaltfläche zur Ausführung anzeigen
button = widgets.Button(description="✅ PDF analysieren")
output = widgets.Output()

def on_button_clicked(b):
    with output:
        output.clear_output()
        if not upload.value:
            print("Bitte zuerst eine PDF-Datei hochladen.")
            return

        print("Starte OCR-Verarbeitung ...")
        ocr_text = run_ocr(upload)
        display(Markdown("### 📃 Extrahierter Text:"))
        display(Markdown(f"```\n{ocr_text[:2000]}\n...```"))  # Nur Vorschau

        df = extract_key_data(ocr_text)
        display(Markdown("### 🔍 Analyse der wichtigsten Vertragsdaten:"))
        display(df)

        # Optional: Exportieren
        df.to_csv("werkvertrag_extrahiert.csv", index=False)
        print("\nErgebnisse gespeichert unter 'werkvertrag_extrahiert.csv'")

button.on_click(on_button_clicked)
display(button, output)


## 📂 PDF-Datei des Werkvertrags hochladen:

FileUpload(value=(), accept='.pdf', description='Upload')

Button(description='✅ PDF analysieren', style=ButtonStyle())

Output()